In [ ]:
"""
File 2: Dick Nielsen - Asquith Filter

Additional filtration of merged bond datasets, FISD Mergent
and TRACE Enhanced Data.

Output: cleaned dataset named ... in parquet format, 
analogous to .s7bdat dataset 

"""

########################### Importing Python Libraries ##########################
import pandas as pd
import numpy as np
import wrds
from datetime import datetime, timedelta
import datetime
import sqlite3                   # for SQL query / merging purposes 

##################### Preliminary Variables/Paths Defined ########################
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/"
download_dir = onedrive_pth + "(draft) adapted_code_bonds_python/"

#################### Connect to WRDS & Download Raw Data Files ####################


In [ ]:
"""
FILE 1: BOND NAMES LIST file
Output: List of bonds that are (1) in TRACE and FISD (TRACE_mergent_names)
(2) PERMCO match via the crsp.msenames file, 
(3) bond-level filters (e.g., bond type, not variable coupon, etc.)

"""

########################### Importing Python Libraries ##########################
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import wrds
import sqlite3                   # for SQL query / merging purposes 
import datetime

##################### Preliminary Variables/Paths Defined ######################## 

onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/"
download_dir = onedrive_pth + "(draft) adapted_code_bonds_python/"

# FISD List of variables
fisd_vars1 = ['issue_id', 'complete_cusip', 'offering_date', 'maturity', 
              'first_interest_date', 'dated_date', 'prospectus_issuer_name', 
              'bond_type', 'issuer_id', 'currency',
              'offering_amt', 'principal_amt', 'rule_144a', 'yankee',
              'coupon_type', 'coupon interest_frequency', 'pay_in_kind']
fisd_vars2 = ['asset_backed', 'convertible', 'exchangeable', 'defaulted', 
              'defeased', 'defeased_date', 'perpetual', 
              'preferred_security', 'putable', 'redeemable', 'security_level', 
              'security_pledge', 'slob', 'coupon_change_indicator']
collist = fisd_vars1 + fisd_vars2

In [ ]:
"""
Step 0: 
Start with FISD list (requiring cusip9 also appears in TRACE data)

"""

#################### Connect to WRDS & Download Raw Data Files ####################
conn = wrds.Connection()

# Get table FISD
bonds = conn.get_table(library='fisd', table='fisd_mergedissue', 
                       columns = collist)

# Drop records missing primary IDs (just in case - not actually needed)
bonds = bonds[(pd.notnull(bonds["issue_id"])==True) &       
              (pd.notnull(bonds['complete_cusip'])==True)]      

# Rename one column 
colmap = {'complete_cusip' : "cusip9"}
bonds = bonds.rename(columns=colmap)

# Get TRACE Enhanced Table From WRDS
trace = conn.get_table(library='trace', table='trace_enhanced_names')

# For industry filter later - industry code from FISD Mergent
fisd_issuer = conn.get_table(library = 'fisd', table = 'fisd_issuer')

# CRSP MSENAMES file 
msenames = conn.get_table(library = 'crsp', table = 'msenames')

#################### 1st Merge: Require CUSIP Match in TRACE ######################

# Set up querying database 
con = sqlite3.connect("fisd.db")
bonds.to_sql("bonds", con, index=False, if_exists='replace')
trace.to_sql("trace", con, index=False, if_exists='replace')

# Query to be submitted
sql = """SELECT DISTINCT a.cusip9, a.*, trace2.cusip_id
        FROM bonds as a
        LEFT JOIN
        (SELECT * FROM trace
        WHERE cusip_id != "" or cusip_id != "None"
        ) AS trace2 
       ON a.cusip9 = trace2.cusip_id 
       WHERE (trace2.cusip_id != "" or trace2.cusip_id != "None");""" 

# Run Query 
bonds = pd.read_sql_query(sql, con)
bonds                         # should yield 215260 rows 

In [ ]:
"""
Step 1: 
Bond-Level Filtration Steps 

After merging, steps are taken to ensure that filters are 
placed on maturity, currencies, amounts, get rid of 
special types, etc.

"""

############################ Fixing Column Types ################################

bonds['offering_date'] = pd.to_datetime(bonds['offering_date'])
bonds['maturity'] = pd.to_datetime(bonds['maturity'])
bonds['first_interest_date'] = pd.to_datetime(bonds['first_interest_date'])
bonds['dated_date'] = pd.to_datetime(bonds['dated_date'])

##################### Maturity, Currency & Domestic, $1M+ #######################

# USD-denominated, no foreign issuer, par value > 1M, no perpetuity
cond_list1 = ((bonds['currency'] == 'USD' or bonds['currency'].isna() == True)
            and (bonds['yankee'] != 'Y') and (bonds['offering_amt'] > 1000)
            and (bonds['maturity'] >= datetime(1, 1, 2012))
            and (bonds['offering_date'] <= datetime(12, 31, 2020)))
bonds1 = bonds[[cond_list]]

############################ Filters on Bond Type ###############################

# Get rid of convertibles/special types/irregular coupons
cond_list2 = ((bonds1['rule_144a'] != "Y")
             and (bonds1['bond_type'] in ['CMTN', 'CMTZ', 'CDEB', 'RNT', 'CZ'])
             and (bonds1['interest_frequency'] in [2, 0, 4, 12, 1])
             and (bonds1['coupon_type'] != 'V')
             and (bonds1['perpetual'] != "Y")
             and (bonds1['convertible'] != "Y")
             and (bonds1['exchangeable'] != "Y")
             and (bonds1['putable'] != "Y")
             and (bonds1['pay_in_kind'] != "Y"))
bonds1 = bonds1[[cond_list2]]

########################## Secured or Asset-Backed ##############################

# Slob: flag denoting that the issue is a secured lease obligation issue
cond_list3 = ((bonds1['preferred_security'] != "Y")
              and (bonds1['security_level'] != "SS")
              and (bonds1['asset_backed'] != "Y")
              and (bonds1['Slob'] != "Y")
              and (bonds1['security_pledge'] != "M")
             )
bonds1 = bonds1[[cond_list3]]

######################### Data Availability Issues ##############################

# Require offering & maturity dates 
cond_list4 = ((bonds1['offering_date'].isna() == False)
             and (bonds1['first_interest_date'].isna() == False)
             or (((bonds1['coupon'].isna() == True) 
                   or (bonds1['first_interest_date'].isna() == True) 
                   or (bonds1['dated_date'].isna() == True)) 
                  and (bonds1['interest_frequency']) == 0))

# Require coupon variables unless zero bond
bonds1 = bonds1[[cond_list4]]

###################### Industry: Financials/Banks Out ###########################

"""proc sql;
	create table bonds1 as
	select distinct l.*, r.Industry_group, r.industry_code
	from bonds1 as l
	left join fisd.fisd_issuer as r
	on l.issuer_id=r.issuer_id
	having r.industry_code not in ("20")  			/* 20=banks */
;quit;"""

# Set up querying database 
con = sqlite3.connect("fisd_issuer.db")
bonds1.to_sql("bonds1", con, index=False, if_exists='replace')
fisd_issuer.to_sql("fisd_issuer", con, index=False, if_exists='replace')

# Submit query - banks have code 20 
sql = """SELECT DISTINCT l.*, r.industry_group, r.industry_code
        FROM bonds1 AS l
        LEFT JOIN
        fisd_issuer AS r
       ON l.issuer_id = r.issuer_id 
       WHERE (r.industry_code != 20);""" 

# Run above query 
bonds1 = pd.read_sql_query(sql, con)
bonds1                   

In [ ]:



"""*--------------------- Maturity>=2022, CURRENCY AND DOMESTIC, $1M+ ------------------;
data bonds1; 
	set bonds; 
	if currency='USD' or missing(currency) ; 
	if yankee^='Y';															/*Yankee = A flag indicating that the issue has been issued by a foreign issuer, but has been registered with the SEC and is payable in dollars.;*/
	if offering_amt>1000;													/* >1million. Offering_amt = The par value of debt initially issued.;*/
	if (maturity >= '01Jan2012'd) and (offering_date <= '31Dec2020'd);		/* Note: only perpetual bonds have missing maturity*/
run;

*--------------------- BOND TYPE ------------------;
data bonds1; 
	set bonds1;
	if rule_144a^='Y'   ; 														/* Kick 144A bonds. Rule_144a= A flag denoting that the issue is a private placement exempt from registration under SEC Rule 144a. Rule 144a issues are generally offered to a limited number of institutional investors;*/
	if bond_type in ('CMTN' 'CMTZ' 'CDEB' 'RNT' 'CZ' )   ;						/* bond_type in  CMTN [US Corporate MTN], CMTZ [US Corporate MTN Zero], CDEB [US Corporate Debentures], RNT [Retail Note], CZ [US Corporate Zero]) */
	if interest_frequency in ('2' '0' '4' '12' '1')   ;							/* More than 99% have this*/
	if coupon_type^='V'   ;														/* Kick variable rate bonds. Are these floating bonds? Their coupon changes with some interest rate (e.g., LIBOR). Not standardized how often and frequently coupon changes. */
	if perpetual^='Y'   ;														/* Kick perpetual */
	if convertible^='Y'   ; 													/* Kick convertible bonds. Convertible = Flag indicating the issue can be converted to the common stock (or other security) of the issuer. Further information can be found in the CONVERTIBLE table.;*/
	if exchangeable ^= 'Y'   ;													/* Kick exchangeable bonds. Can be exchanged for securities of other firms (usually subsidiary) */
	if putable ^= 'Y'   ;														/* Kick putable bonds. They have a 'put option' forcing early repayment */
	if pay_in_kind^='Y'; 														/* Kick pay_in_kind: means that interest can be paid by giving more of the same bond issue or by giving other securities instead of cash. .;*/	
run;

*--------------------- SECURED OR ASSET-BACKED? ------------------;
data bonds1; 
	set bonds1;
	if preferred_security ^='Y'   ; 											/* Kick preferred_securities. Preferred_security = Flag indicating this issue is a preferred security;*/
	if security_level^=SS   ; 													/* SS= senior secured. We skip it because some are in the Hotchkiss treatment sample.*/
	if asset_backed^='Y'   ; 													/* Kick asset_backed bonds. Asset_backed = Flag indicating that the issue is an asset-backed issue, that is collateralized by a portfolio of loans or assets other than single family mortgages. */
	if Slob^='Y'   ; 															/* if Slob^=Y. Slob = flag denoting that the issue is a secured lease obligation issue (i.e., an issue secured by one or more leases issued in a sales leaseback transaction by an electric utility). Kick security_pledge bonds. */
	if security_pledge^='M'   ; 												/* Kick security_pledge bonds. Security_pledge = A flag indicating that certain assets have been pledged as security for the issue. Can be missing, M [mortgate], or N [Note].*/
run;

*--------------------- DATA AVAILABILITY ------------------;
data bonds1; 
	set bonds1;
	if not cmiss(offering_date,maturity);
	if cmiss(coupon,first_interest_date, dated_date) and interest_frequency^='0' then delete;	/* require coupon-variables UNLESS zero bond */
run;

*--------------------- INDUSTRY CODE: Kick banks ------------------;
** Include industry_type;
proc sql;
	create table bonds1 as
	select distinct l.*, r.Industry_group, r.industry_code
	from bonds1 as l
	left join fisd.fisd_issuer as r
	on l.issuer_id=r.issuer_id
	having r.industry_code not in ("20")  			/* 20=banks */
;quit;
"""

In [ ]:
"""
Step 2: 
Include Permco from crsp.msenames on ncusip6, which contains 
{ncusip6, permco}-combinations;

"""

# Change column types in MSENAMES
msenames['namedt'] = pd.to_datetime(msenames['namedt'])
msenames['nameendt'] = pd.to_datetime(msenames['nameendt'])

"""
proc sql;
	create table bonds2 as
	select distinct l.cusip9, r.permco, l.*	
					, ( min(r.namedt) <= l.offering_date <= max(r.nameendt) ) as d_offering_covered
					, (not missing(permco)) as d_permco
						/* , l.issue_id , l.maturity, l.prospectus_issuer_name
						, min(r.namedt) as start_cusip6_permco
						, max(r.nameendt) as end_cusip6_permco */
	from bonds1 as l
	left join crsp.msenames as r
	on substr(l.cusip9,1,6)=substr(r.ncusip,1,6)
	group by substr(r.ncusip,1,6) , permco					/* for each cusip6-permco */
	/*having ( not cmiss(l.cusip9,permco))*/
;quit;
"""

# Set up querying database 
con = sqlite3.connect("crsp.db")
bonds1.to_sql("bonds1", con, index=False, if_exists='replace')
msenames.to_sql("msenames", con, index=False, if_exists='replace')

# Query to be submitted
sql = """SELECT DISTINCT l.cusip9, r.permco, l.*, 
        ( min(r.namedt) <= l.offering_date <= max(r.nameendt) ) 
        AS d_offering_covered, 
        (permco IS NOT NULL) as d_permco
        FROM bonds1 as l
        LEFT JOIN
        msenames as r
       ON substr(l.cusip9,1,6)=substr(r.ncusip,1,6)
       GROUP BY substr(r.ncusip, 1, 6), r.permco;""" 

# Run Query 
bonds2 = pd.read_sql_query(sql, con)
bonds2                         # 

In [ ]:
"""
Save, reformat columns, and output as parquet file: 
bond_list_permco.parquet. 

Parquet saves space! 

"""

bonds2.to_parquet(download_dir + "bond_list_permco.parquet")